In [1]:
import sys
sys.path.append('/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/analysis/')

from functions import *

In [2]:
def get_resids(data, var):
    import statsmodels.formula.api as smf
    model = smf.ols(var + '~ interview_age + C(sex)', data=data)
    model = model.fit()
    resid = model.resid
    return resid

def add_resid(data, var):
    data[var + "_r"] = get_resids(data, var)
    return data

In [3]:
def data_desc(url, url2=None, pull_list=None):
    
    dict_df = pd.read_html(url)[3]
    
    if url2 is not None:
        dict_df2 = pd.read_html(url2)[3]
        dict_df = pd.concat([dict_df, dict_df2])
    
    output = (dict_df[['Element Name','Description']]
     .rename({'Element Name':'Name'}, axis=1)
    )
    
    if pull_list is not None:
        output = output.query('Name not in @pull_list')
        
    return output.reset_index(drop=True)
    #return dict_df
    
strings_to_filter = ['subjectkey','src_subject_id','interview_date','interview_age','sex','eventname']

In [4]:
pd.options.display.max_colwidth = 500

nback_var_dict = data_desc('https://nda.nih.gov/data_structure.html?short_name=abcd_mrinback02',
                           pull_list = strings_to_filter)


In [5]:
remove = ['Standard deviation', 'switch', 'trials', 'visit']
pull = ['correct responses']
nback_vars = (nback_var_dict.fillna('')
 .query('not Description.str.contains("|".join(@remove))', engine='python')
 .query('Description.str.contains("|".join(@pull))', engine='python')
).Name.to_list()

In [6]:
bpath = '/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/behavioral/'

df_nback = (readtxt(bpath + 'abcd_mrinback02.txt')[['src_subject_id', 'eventname', 'interview_age', 'sex'] + nback_vars]
            .query('eventname == "baseline_year_1_arm_1"'))

df_nback['ID'] = df_nback['src_subject_id'].str.replace('_', '', regex=True)

In [7]:
df_nback=df_nbackdf_nback = df_nback[['ID', 'tfmri_nb_all_beh_ctotal_nt', 'tfmri_nb_all_beh_ctotal_rate','tfmri_nb_all_beh_ctotal_mrt', 
                                      'tfmri_nb_all_beh_c2b_rate', 'interview_age', 'sex']]

In [8]:
df_nback[['interview_age', 'tfmri_nb_all_beh_ctotal_mrt','tfmri_nb_all_beh_ctotal_rate','tfmri_nb_all_beh_ctotal_nt',  'tfmri_nb_all_beh_c2b_rate']] = (df_nback[['interview_age', 'tfmri_nb_all_beh_ctotal_mrt','tfmri_nb_all_beh_ctotal_rate','tfmri_nb_all_beh_ctotal_nt',  'tfmri_nb_all_beh_c2b_rate']]
                                                                                                                          .apply(pd.to_numeric, errors='coerce'))

In [9]:
nback_subscales=['tfmri_nb_all_beh_ctotal_mrt','tfmri_nb_all_beh_ctotal_rate','tfmri_nb_all_beh_ctotal_nt',  'tfmri_nb_all_beh_c2b_rate']

for i in nback_subscales: 
    df_nback = add_resid(df_nback, i)
    
nback_sub_scales = [nback_subscales + "_r" for nback_subscales in nback_subscales]
df_nback = df_nback[['ID'] + nback_sub_scales]

In [10]:
df_nback.columns = ['ID', 'nb_total_n_correct', 'nb_total_rate_correct', 'nb_mean_rt_corect', 'nb_all_beh_c2b_rate']
df_nback.to_csv(bpath + 'nback_behavior_baseline.csv', index=False)